# Café Sales Data Analysis - Data Cleaning & Preparation

## Executive Summary
This notebook provides comprehensive data cleaning and preparation for café sales analysis. We process 10,000 transactions from 2023 to ensure data quality and reliability for subsequent business intelligence analysis. The cleaning process addresses missing values, data type inconsistencies, and implements strategic imputation methods based on statistical analysis.


## Data Quality Objectives
1. **Completeness**: Address missing values using statistically sound imputation methods
2. **Consistency**: Standardize data types and formats across all columns
3. **Accuracy**: Validate business logic and recalculate derived fields
4. **Reliability**: Remove or flag problematic records that could skew analysis

---

## 1. Environment Setup and Data Import

### 1.1 Library Imports
We import essential libraries for data manipulation and analysis. These form the foundation of our data cleaning pipeline.

In [1]:
# Import essential libraries for data cleaning and analysis
import pandas as pd  # Data manipulation and analysis
import numpy as np   # Numerical computing and array operations
import matplotlib.pyplot as plt  # Data visualization (for validation plots if needed)

# Configure pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

### 1.2 Data Import and Initial Exploration

**Strategy**: We use strategic NA value handling during import to catch common data quality issues:
- "UNKNOWN" values indicate missing product information
- "ERROR" values suggest system failures during data collection
- Empty strings represent incomplete form submissions

This approach allows us to identify and quantify data quality issues from the start.

In [14]:
# Load the raw café sales data with comprehensive NA value detection
# Note: Adjust the file path as needed for your environment
df = pd.read_csv('/Users/kabbo/Desktop/marcy/Project 2 Cafe Sales/M1-FINAL-PROJECT-THIERNO_KABBO/data/raw/cafe_sales.csv', 
                 na_values=["UNKNOWN", "ERROR", ""])

# Initial data exploration to understand the dataset structure
print(f"Dataset dimensions: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Total transactions: {df.shape[0]:,}")
print(f"Date range coverage: {df['Transaction Date'].nunique()} unique dates")



Dataset dimensions: 10,000 rows × 8 columns
Total transactions: 10,000
Date range coverage: 365 unique dates


## 2. Data Quality Assessment

### 2.1 Data Types and Structure Analysis

Understanding the current data types helps us identify necessary conversions and potential issues.

In [ ]:
#  Data structure analysis
df.info()

for col in df.columns:
    print(f"  • {col}: {df[col].dtype}")

for col in df.columns:
    unique_count = df[col].nunique()
    print(f"  • {col}: {unique_count:,} unique values")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              9031 non-null   object 
 2   Quantity          9521 non-null   float64
 3   Price Per Unit    9467 non-null   float64
 4   Total Spent       9498 non-null   float64
 5   Payment Method    6822 non-null   object 
 6   Location          6039 non-null   object 
 7   Transaction Date  9540 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB
  • Transaction ID: object
  • Item: object
  • Quantity: float64
  • Price Per Unit: float64
  • Total Spent: float64
  • Payment Method: object
  • Location: object
  • Transaction Date: object
  • Transaction ID: 10,000 unique values
  • Item: 8 unique values
  • Quantity: 5 unique values
  • Price Per Unit: 6 unique values
  • Total Spent: 17 unique values
  • Pa

### 2.2 Missing Values Analysis

**Business Impact**: Missing values can significantly impact analysis accuracy. We analyze patterns to understand:
- Whether missing values are random or systematic
- Which columns are most affected
- Potential business reasons for missing data

In [15]:
# Detailed missing values analysis 
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_summary = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent.round(2)
})

# Sort by missing percentage to prioritize cleaning efforts
missing_summary = missing_summary.sort_values('Missing Percentage', ascending=False)
print(missing_summary)


                  Missing Count  Missing Percentage
Location                   3961               39.61
Payment Method             3178               31.78
Item                        969                9.69
Price Per Unit              533                5.33
Total Spent                 502                5.02
Quantity                    479                4.79
Transaction Date            460                4.60
Transaction ID                0                0.00


### 2.3 Duplicate Records Analysis

**Methodology**: We check for duplicates across key business fields to identify potential data entry errors or system issues.

In [16]:


duplicate_rows = df[df.duplicated(subset=['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Transaction Date'], keep=False)]
print("Duplicate Rows:")
print(duplicate_rows)

num_duplicates = df.duplicated(subset=['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Transaction Date'], keep=False).sum()
print(f"Number of duplicate rows: {num_duplicates}")


Duplicate Rows:
Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date]
Index: []
Number of duplicate rows: 0


## 3. Data Cleaning Implementation

### 3.1 Item Column Cleaning Strategy

**Business Decision**: We preserve "UNKNOWN" items rather than imputing them because:
1. They represent real business scenarios (unscanned items, manual entries)
2. Imputation could mask operational issues that need addressing
3. They can be analyzed separately to understand system gaps

In [23]:
# Strategic handling of missing Item values

# Convert NaN values to 'UNKNOWN' for explicit tracking
df['Item'] = df['Item'].fillna('UNKNOWN')

# Analyze item distribution after cleaning
item_distribution = df['Item'].value_counts()
print("Item distribution after cleaning:")
print(item_distribution)

# Calculate percentage of unknown items
unknown_percentage = (item_distribution['UNKNOWN'] / len(df)) * 100
print("Unknown Percentage: ", unknown_percentage)


Item distribution after cleaning:
Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      969
Name: count, dtype: int64
Unknown Percentage:  9.69


### 3.2 Quantity Imputation Strategy

**Statistical Approach**: Based on preliminary analysis (referenced from Excel analysis), the Quantity distribution shows slight skewness. We use median imputation because:
1. **Robust to outliers**: Median is less affected by extreme values
2. **Business logic**: Most café transactions involve 1-3 items
3. **Conservative approach**: Avoids inflating average transaction sizes

In [39]:
# Analyze quantity distribution before imputation
print("Quantity statistics before imputation:")
print(df['Quantity'].describe())

# Calculate imputation value
quantity_median = df['Quantity'].median()
missing_quantity_count = df['Quantity'].isnull().sum()

print(f"\nImputation details:")
print(f"• Missing values: {missing_quantity_count:,} ({missing_quantity_count/len(df)*100:.1f}%)")
print(f"• Median value used for imputation: {quantity_median}")
print(f"• Rationale: Median chosen due to skewed distribution (robust to outliers)")

# Perform imputation
df['Quantity'] = df['Quantity'].fillna(quantity_median)

# Convert to integer for business logic (can't have fractional items)
df['Quantity'] = df['Quantity'].astype(int)



Quantity statistics before imputation:
count    10000.000000
mean         3.027100
std          1.384614
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max          5.000000
Name: Quantity, dtype: float64

Imputation details:
• Missing values: 0 (0.0%)
• Median value used for imputation: 3.0
• Rationale: Median chosen due to skewed distribution (robust to outliers)


### 3.3 Price Per Unit Imputation Strategy

**Statistical Approach**: Price Per Unit shows normal distribution characteristics, making mean imputation appropriate:
1. **Central tendency**: Mean represents typical pricing
2. **Business continuity**: Maintains overall revenue patterns
3. **Minimal bias**: Normal distribution supports mean imputation

In [40]:
# Analyze price distribution before imputation
print("Price Per Unit statistics before imputation:")
print(df['Price Per Unit'].describe())

# Calculate imputation value
price_per_unit_mean = df['Price Per Unit'].mean()
missing_price_count = df['Price Per Unit'].isnull().sum()

print(f"\nImputation details:")
print(f"• Missing values: {missing_price_count:,} ({missing_price_count/len(df)*100:.1f}%)")
print(f"• Mean value used for imputation: ${price_per_unit_mean:.2f}")
print(f"• Rationale: Mean chosen due to normal distribution of prices")

# Perform imputation
df['Price Per Unit'] = df['Price Per Unit'].fillna(price_per_unit_mean)


Price Per Unit statistics before imputation:
count    10000.000000
mean         2.949984
std          1.243910
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max          5.000000
Name: Price Per Unit, dtype: float64

Imputation details:
• Missing values: 0 (0.0%)
• Mean value used for imputation: $2.95
• Rationale: Mean chosen due to normal distribution of prices


### 3.4 Total Spent Recalculation

**Business Logic**: Rather than imputing Total Spent, we recalculate it to ensure data consistency:
1. **Data integrity**: Eliminates discrepancies between components and totals
2. **Audit trail**: Clear calculation method for financial validation
3. **Business accuracy**: Ensures all financial metrics are mathematically correct

In [ ]:
# Recalculate Total Spent
df['Total Spent'] = df['Price Per Unit'] * df['Quantity']


# Checking and comparing with Excel results 
print(df[['Item', 'Quantity', 'Price Per Unit', 'Total Spent']].head(10))

       Item  Quantity  Price Per Unit  Total Spent
0    Coffee         2             2.0          4.0
1      Cake         4             3.0         12.0
2    Cookie         4             1.0          4.0
3     Salad         2             5.0         10.0
4    Coffee         2             2.0          4.0
5  Smoothie         5             4.0         20.0
6   UNKNOWN         3             3.0          9.0
7  Sandwich         4             4.0         16.0
8   UNKNOWN         5             3.0         15.0
9  Sandwich         5             4.0         20.0


### 3.5 Categorical Data Handling Strategy

**Business Decision**: For Payment Method and Location, we preserve missing values because:
1. **Operational insights**: Missing data patterns may indicate system issues
2. **Customer behavior**: Some customers may prefer not to specify location
3. **Data integrity**: Imputation could mask important business problems

These fields will be analyzed separately to understand missing data patterns.

### 3.6 Transaction Date Cleaning

**Critical Business Decision**: Transaction dates are essential for time-series analysis. Missing dates cannot be reliably imputed without introducing bias, so we remove these records:
1. **Temporal analysis integrity**: Date is crucial for trend analysis
2. **Business reporting**: All financial reports require valid dates
3. **Data quality**: Better to have complete records than imputed dates

In [50]:
df = df.dropna(subset=['Transaction Date'])

print(df['Transaction Date'].isnull().sum())  # Should print 0
print(df['Transaction Date'].count())        # See all unique values



0
9540


In [ ]:
#Exporting cleaned data for next steps

df.to_csv('cleaned_cafe_sales.csv', index=False)